In [251]:
import tensorflow as tf
import numpy as np
from common import *

from asyncio.events import  AbstractEventLoop
import logging
import os, time
from numba import jit
import tflearn
from tflearn.layers.core import time_distributed
from tflearn.layers import conv_2d
from io import StringIO

logging.basicConfig(level=logging.DEBUG)
os.environ['PYTHONASYNCIODEBUG'] = '1'

In [351]:
def parse_csv(text):
    strings = tf.string_split([text], delimiter='\n')
    raw_nums = tf.string_split(strings.values)
    nums = tf.string_to_number(raw_nums.values, tf.int32)

    dense = tf.sparse_to_dense(raw_nums.indices, 
                               raw_nums.dense_shape, 
                               nums,
                               default_value=0)
#     dense.set_shape(raw_nums.get_shape())
    return dense


def read_input_tuple(filename_queue):
    fnames = filename_queue.dequeue()
    for fn in tf.unstack(fnames):
        record_string = tf.read_file(fn)
#         example = parse_csv(record_string)
#     processed_example = some_processing(example)
#     return example, key
    return record_string


def input_pipeline(filenames, batch_size, num_epochs=None):
    string_tensor = tf.convert_to_tensor(filenames, dtype=tf.string)
    string_tensor = tf.identity(string_tensor)
    filename_queue = tf.train.input_producer(
        filenames, 
#         element_shape=[],
        num_epochs=num_epochs, 
        capacity=32,
        shuffle=False,
        seed=0)
    fn = read_input_tuple(filename_queue)    

#     min_after_dequeue = 10000
#     capacity = min_after_dequeue + 3 * batch_size
#     example_batch, label_batch = tf.train.batch(
#         [example, key], 
#         batch_size=batch_size, 
#         capacity=capacity,
# #         min_after_dequeue=min_after_dequeue,
#         dynamic_pad=True,
#         allow_smaller_final_batch=True,
#         num_threads=cpu_count
#     )
#     return example_batch, label_batch

    return fn

In [208]:
filenames = glob('../data/corpus/*.txt')[100:112]
filenames = np.reshape(filenames, (-1,3))

In [209]:
filenames

array([['../data/corpus/5984bf58b6b11307a2638529.txt',
        '../data/corpus/5984c732b6b113644f638532.txt',
        '../data/corpus/5984bd1eb6b1136c756384f5.txt'],
       ['../data/corpus/5984c1fdb6b1132b6863853e.txt',
        '../data/corpus/5984cfd2b6b113399a6384f7.txt',
        '../data/corpus/5984c4eeb6b1134ab8638530.txt'],
       ['../data/corpus/5984cb0ab6b1130b9863852c.txt',
        '../data/corpus/5984d818b6b1130a266384f1.txt',
        '../data/corpus/5984dbd1b6b113237863853f.txt'],
       ['../data/corpus/5984c861b6b1136d74638511.txt',
        '../data/corpus/5984c20db6b1132c8e6384f7.txt',
        '../data/corpus/5984bb6db6b11359be638527.txt']],
      dtype='<U43')

In [247]:
filenames.shape

(4, 3)

In [352]:
start_time = time.time()

g = tf.Graph()
with g.as_default():  
    tf.set_random_seed(0)
    sess = tf.Session()
    with sess.as_default():
#         example_batch, label_batch = input_pipeline(filenames, batch_size=2, num_epochs=1)      
        res = input_pipeline(filenames, batch_size=2, num_epochs=1)        
        
        init_local = tf.local_variables_initializer()
        init_global = tf.global_variables_initializer()
        sess.run([init_global, init_local])
                
        # Start input enqueue threads.
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)

        try:
            while not coord.should_stop():
#                 batches, keys = sess.run([example_batch, label_batch])  
                [fnq] = sess.run([res])  
#                 time_distributed(batches, conv_2d, [num_filters, filter_sizes, strides])
                print(fnq)
        except tf.errors.OutOfRangeError:
            print('Done training -- epoch limit reached')
        finally:
            # When done, ask the threads to stop.
            coord.request_stop()

        # Wait for threads to finish.
        coord.join(threads)        

print("--- %s seconds ---" % (time.time() - start_time))

b'2 110 92 3935 2690 1109 1730\n2 110 92 3935 2690 1109 1730 4174\n17 213 179 607 118 763 193 2487 3979 1 399 617 8 462 6701 462 2228 3979 5162 732 502 1459 22514 769 3979 2487 1988 1027 16112 820 1756 469 732 502\n'
b'4 549 1498 1670 1204 3434 120629 40 2 240 21960 1190\n4 549 1498 1670 1204 3434 120629 40 254 1190 18 197 1639 43 254 1190 110 1465 43 254 1190 1809 206 2229 196 601 2251 1946 2251 1449 1031 15384 40 71 1616 17 1809 206 2229 196\n4 17 197 1639 43 254 1190 23 768\n4 17 1465 43 254 1190 55 77 190 87 522 768 575 1946 2229 196\n4 17 1946 2251 23 615 196\n4 17 1946 2251 23 9227\n2 240 21960 1190 18 2251 1449 1031 120629 40 71 573 41 3434 2229 196 601 2251 6 2909 179 43 254 1190 17 165 5 615 196 1946 2251 45 6 2909 179 43 23 768 711 221\n2 17 45 615 196 23 9227 4677 1078\n'
b'2 5866 747 141 1190\n104 236 209 95 105 2 5866 141 1168 1190 2154 6376 13420 1498 47401 24983 2060 26259 320 13127 17515 6034\n2 5866 747 141 1190 77 1891 2802 18713 134 5568 5 842 119 405 3042 119 286 15

# My input func

In [166]:
num_threads=cpu_count

def input_pipeline(filenames, batch_size, num_epochs=None):    
    filename_queue = tf.train.string_input_producer(
      filenames, num_epochs=num_epochs, shuffle=False, seed=0)
    example_queue, key = read_input_file(filename_queue)    

    # Create a training graph that starts by dequeuing a batch of examples.
    inputs = example_queue.dequeue_up_to(batch_size)
#     train_op = ...use 'inputs' to build the training part of the graph...

    
#     min_after_dequeue = 10000
#     capacity = min_after_dequeue + 3 * batch_size
#     example_batch, label_batch = tf.train.batch(
#         [example, key], 
#         batch_size=batch_size, 
#         capacity=capacity,
# #         min_after_dequeue=min_after_dequeue,
#         dynamic_pad=True,
#         allow_smaller_final_batch=True,
#         num_threads=cpu_count
#     )
#     return example_batch, label_batch
    return inputs


g = tf.Graph()
with g.as_default():  
    tf.set_random_seed(0)
    sess = tf.Session()
    with sess.as_default():
        inputs = input_pipeline(filenames, batch_size=2, num_epochs=1)        
        
        init_local = tf.local_variables_initializer()
        init_global = tf.global_variables_initializer()
        sess.run([init_global, init_local])
                
        # Start input enqueue threads.
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)

        try:
            while not coord.should_stop():
                examples = sess.run([inputs, label_batch])  
#                 time_distributed(batches, conv_2d, [num_filters, filter_sizes, strides])
                pprint(keys)
        except tf.errors.OutOfRangeError:
            print('Done training -- epoch limit reached')
        finally:
            # When done, ask the threads to stop.
            coord.request_stop()

        # Wait for threads to finish.
        coord.join(threads)        

AttributeError: 'Tensor' object has no attribute 'dequeue_up_to'

In [340]:
g = tf.Graph()
with g.as_default():  
    tf.set_random_seed(0)
    sess = tf.Session()
    with sess.as_default():
        fn = tf.encode_base64(b'../data/corpus/5984d818b6b1130a266384f1.txt')
#         res = tf.read_file(fn)
        [cont] = sess.run([fn])
        print(cont)


b'Li4vZGF0YS9jb3JwdXMvNTk4NGQ4MThiNmIxMTMwYTI2NjM4NGYxLnR4dA'
